In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
# import flask
# from flask import Flask
# from flask import request
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.6 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=646f75649e30c858338bfe1439c49ec6f0f4144590c2e8f04e4726d46ae7e6c9
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.8 

In [5]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [6]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [7]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [8]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [9]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [10]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [11]:
wordLemm = WordNetLemmatizer()
def processed_paragraph(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [12]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [13]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
new_df = generate_new_df(df)

In [16]:
new_df_validation = generate_new_df(df_validation)

In [17]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [18]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [19]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
questions= new_df['postText']
answers= new_df['spoiler']
stories_not_summarized = new_df['paragraph']
tags = new_df['tags']

In [21]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

# from transformers import BartTokenizer, BartModel #, BartForCausalLM #,  BartModel

# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# model = BartModel.from_pretrained('facebook/bart-large')

In [ ]:
# summarize with BART
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn")

In [22]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-t5-base" # you can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

In [23]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=f44f0cc7c0bebf0716fa1df5a22b51a1af5b71b2b1e19be251ab37fcd246b9ae
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [24]:
# from transformers import pipeline
# import requests
# import pprint
# import time
# pp = pprint.PrettyPrinter(indent=14)

# from transformers import T5Tokenizer, T5ForConditionalGeneration
import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-base")
# model = BartModel.from_pretrained('facebook/bart-large')

In [25]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
from datetime import datetime

In [27]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [28]:
from tqdm.autonotebook import tqdm
total1 = len(stories_not_summarized)
# total1 = len(40)
summary_manifestos_t5 = []
for i in tqdm(range(total1)): 
    stories= stories_not_summarized[i][:1000]
    # summary_manifesto_t5 = summarizer_t5(stories, min_length=4, max_length=500)
    # summary_manifesto_bart = summarizer(input_text, min_length=4, max_length=15) # change min_ and max_length for different output


    if tags[i] == "phrase":
      # summary_manifesto_bart = summarizer_bart(stories, min_length=4, max_length=15) # change min_ and max_length for different output
      summary_manifesto_bart = run_model(stories)
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)
    elif tags[i] == "passage":
      # summary_manifesto_bart = summarizer_bart(stories, min_length=15, max_length=50) # change min_ and max_length for different output
      summary_manifesto_bart = run_model(stories)
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)
    elif tags[i] == "multi":
      # summary_manifesto_bart = summarizer_bart(stories, min_length=50, max_length=500) # change min_ and max_length for different output
      summary_manifesto_bart = run_model(stories)
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)

    # summary_manifesto_bart = summarizer_bart(stories, min_length=4, max_length=500) # change min_ and max_length for different output
    # summary_manifestos_t5.append(summary_manifesto_bart)
    # print("\n")
    # print(summary_manifesto_bart)
    # print("\n")
    # print(tags[i])

    # summary_manifestos_t5.append(summary_manifesto_bart)

file = open("UnifiedQA_results.txt", "w")

#convert variable to string
str_1 = repr(summary_manifestos_t5)
file.write("UnifiedQA_results = " + str_1 + "\n")

#close file
file.close()

  0%|          | 0/3200 [00:00<?, ?it/s]

["welker was one of brady's favorite targets for six seasons from"]
['the recovery of the ozone layer is not expected to be based on the chlorine decline']
['intellectual stimulation is the most important aspect of employee happiness']
['follow your passion']
['it only takes one piece of equipment']
['no']
["they're too bulky"]
['the temperature was around 10 degrees celsius']
['red wine is associated with lower cardiovascular risk']
['we tried 29 different kinds of hot sauce to find our favorite']
['a percentage of income']
['yes']
['some coupon codes will let you know if there are any available at the store']
['target is all about inclusion']
['the newest fad in china is glass pedestrian bridges']
['pinch and zoom feature']
['the boston city subway is a germaphobes nightmare the ticketvending touchscreen']
['there are only 12 gryffindors in harrys year']
['the camera is hidden behind the infotainment screen.']
['there is a boyfriend and she plays around on the side.']
["anjelica aj h

In [29]:
stories = summary_manifestos_t5

In [30]:
type(stories[0])

list

In [31]:
stories_list =[]
for i in range(len(stories)):
    summary=stories[i][0]
    stories_list.append(summary)


print(stories_list)

["welker was one of brady's favorite targets for six seasons from", 'the recovery of the ozone layer is not expected to be based on the chlorine decline', 'intellectual stimulation is the most important aspect of employee happiness', 'follow your passion', 'it only takes one piece of equipment', 'no', "they're too bulky", 'the temperature was around 10 degrees celsius', 'red wine is associated with lower cardiovascular risk', 'we tried 29 different kinds of hot sauce to find our favorite', 'a percentage of income', 'yes', 'some coupon codes will let you know if there are any available at the store', 'target is all about inclusion', 'the newest fad in china is glass pedestrian bridges', 'pinch and zoom feature', 'the boston city subway is a germaphobes nightmare the ticketvending touchscreen', 'there are only 12 gryffindors in harrys year', 'the camera is hidden behind the infotainment screen.', 'there is a boyfriend and she plays around on the side.', "anjelica aj hadsell's mom jennife

In [ ]:
# from transformers import pipeline
# qa_pipleline = pipeline('question-answering', model=model, tokenizer=tokenizer, device=0)

The model 'T5ForConditionalGeneration' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'LongformerForQuestionAnswerin

In [ ]:
answers =[]
for i in tqdm(range(len(questions))):
  answer = qa_pipleline({
       'question': questions[i],
        'context': stories_list[i]
  })
  answers.append(answer)

In [ ]:
ans=[]
for i in range(len(answers)):
  ans.append(answers[i]['answer'])
  print(answers[i]['answer'])

In [32]:
type(new_df['spoiler'][0:40])

pandas.core.series.Series

In [33]:
type(stories_list[0:40])

list

In [34]:
stories_list_series = pd.Series((v for v in stories_list))

In [35]:
stories_list_series

0       welker was one of brady's favorite targets for...
1       the recovery of the ozone layer is not expecte...
2       intellectual stimulation is the most important...
3                                     follow your passion
4                    it only takes one piece of equipment
                              ...                        
3195    it' s still not 100 clear what kind of ad inve...
3196                  a mother and her two young children
3197                                               stress
3198    beyoncé is gon na announce a pregnancy in febr...
3199    charlie dent rpa came out in support of gay ma...
Length: 3200, dtype: object

In [36]:
dataframe = pd.DataFrame({ 'postText': questions, 'actual_spoiler':new_df['spoiler'], 'gen_spoiler':stories_list_series })

In [37]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [38]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [39]:
bleu_scores

[4.071258173524672e-155,
 0,
 0.11943296826671962,
 0.025592247818728167,
 1.148548497143447e-78,
 6.689869579535845e-232,
 2.2017045318079177e-78,
 0.2614799220019236,
 0.10579587111633648,
 0.07302697820772387,
 0.24966148140106198,
 1.384292958842266e-231,
 0.008816795319277029,
 0.09081625231038641,
 0.0919922760368168,
 0.0675319167498955,
 3.6167538711641e-78,
 2.8007909588022472e-78,
 0.2499031935858321,
 0.2353614480282375,
 0.30913779572949396,
 3.4535805554687395e-78,
 0.23959617644809125,
 1.2657563801897163e-78,
 0.06272471060479574,
 0.05997570962559865,
 0.1640595257467205,
 0.5683199227383049,
 0.07752142388307205,
 0.02571458701926945,
 1.5918241974932317e-159,
 6.015421429356243e-155,
 1.6131683294950038e-155,
 8.830351914603804e-155,
 0.5732563162650223,
 2.9455919518738093e-155,
 7.397243246681167e-233,
 0.3223348917097043,
 0.0009103774151802314,
 1.651232885718337e-78,
 0.19400004313218128,
 2.662952480522576e-78,
 0.12160403609372213,
 0.09817377146741582,
 7.0074

In [40]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.11

In [41]:
bleu_min = min(bleu_scores)
bleu_min

0

In [42]:
bleu_max = max(bleu_scores)
bleu_max

1.0

In [43]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.005365835356602778

In [44]:
import sys
from nltk.translate.meteor_score import meteor_score

In [45]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [46]:
meteor_scores

[0.0,
 0.0,
 0.20380434782608695,
 0.12195121951219512,
 0.0,
 0.0,
 0.0,
 0.13513513513513514,
 0.3289473684210526,
 0.078125,
 0.0,
 0.0,
 0.0364963503649635,
 0.08064516129032258,
 0.09090909090909091,
 0.07692307692307693,
 0.0,
 0.0,
 0.2427184466019418,
 0.2611940298507463,
 0.12820512820512822,
 0.0,
 0.2922453703703704,
 0.029411764705882353,
 0.03571428571428571,
 0.052631578947368425,
 0.125,
 0.6465763875643715,
 0.05050505050505051,
 0.07042253521126761,
 0.0,
 0.04098360655737705,
 0.0,
 0.0,
 0.9197099892588615,
 0.0,
 0.0,
 0.14880952380952384,
 0.05747126436781609,
 0.0,
 0.13793103448275862,
 0.0,
 0.09433962264150944,
 0.10948905109489049,
 0.0,
 0.04854368932038835,
 0.0,
 0.6465517241379309,
 0.0,
 0.02873563218390805,
 0.0625,
 0.0,
 0.25337837837837834,
 0.04672897196261682,
 0.0,
 0.0,
 0.0,
 0.05494505494505494,
 0.08264462809917356,
 0.0,
 0.06097560975609756,
 0.0,
 0.0,
 0.5597014925373134,
 0.0,
 0.11235955056179774,
 0.4121863799283154,
 0.3982142857142857,

In [47]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.14

In [48]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.9997724169321802

In [49]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [50]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.0364963503649635

In [ ]:
!pip install bert-score

In [ ]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

In [ ]:
bert_scores_P = []
bert_scores_R = []
bert_scores_F1 = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)

  P, R, F1 = score(hypo, ref, lang="en", verbose=True)
  bert_scores_P.append(P)
  bert_scores_R.append(R)
  bert_scores_F1.append(F1)

  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  # meteor_scores.append(meteor)

In [ ]:
# P, R, F1 = score(cands, refs, lang="en", verbose=True)

In [ ]:
bert_scores_P

In [ ]:
bert_scores_R

In [ ]:
bert_scores_F1